In [3]:
import cv2
from ultralytics import YOLO
import pandas as pd
import time

# Load YOLOv8 model
try:
    model = YOLO('yolov8m.pt')
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Path to the input video file
video_path = "C:\\Users\\Vignesh\\Videos\\CAR.mp4"
# Path to the Excel file for storing data
excel_path = "C:\\Users\\Vignesh\\Desktop\\CAR COUNT\\Data for car.xlsx"

# Open the video file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error opening video file")
    exit()

# Set frame rate and create VideoWriter for output
frame_rate = 30
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_path = "C:\\Users\\Vignesh\\Videos\\CAR_detected.avi"
out = cv2.VideoWriter(output_path, fourcc, frame_rate, (int(cap.get(3)), int(cap.get(4))))

# Initialize line position and count
line_position = int(cap.get(4) * 0.7)  # Set line position to 70% of frame height
car_count = 0.0  # Initialize car_count as a float
cars_crossed = {}

# Create an empty DataFrame for the final car count
df = pd.DataFrame(columns=['Total Counter'])  # Only store final car count

# Frame skipping configuration
frame_skip = 2  # Process every 2nd frame
frame_counter = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process only every nth frame
    if frame_counter % frame_skip == 0:
        results = model(frame)

        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = box.conf[0]
                cls = box.cls[0]
                label = model.names[int(cls)]

                # Count only cars (ignore other objects)
                if label == 'car' and conf > 0.7:
                    # Calculate the centroid of the bounding box
                    centroid_y = (y1 + y2) // 2

                    # Draw bounding boxes and labels
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                    # Count the vehicle if the centroid crosses the line downwards
                    if centroid_y > line_position and (x1, y1, x2, y2) not in cars_crossed:
                        cars_crossed[(x1, y1, x2, y2)] = True  # Track the car by its bounding box
                        car_count += 1.0  # Increment car_count as a float

                        # Update the DataFrame with the final count
                        df.loc[0] = [int(car_count / 4)]  # Store the final car count

                        print(f"Total Count: {int(car_count / 4)}")

        # Draw the line for counting
        cv2.line(frame, (0, line_position), (frame.shape[1], line_position), (0, 0, 255), 2)

    # Display updated car count on the frame with integer value
    cv2.putText(frame, f'Car Count: {int(car_count / 4)}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Write the frame to output video
    out.write(frame)

    # Display the resulting frame
    cv2.imshow('Vehicle Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save DataFrame with only the last car count to Excel after processing
if not df.empty:
    df.to_excel(excel_path, index=False)

# Release video writer and capture
out.release()
cap.release()
cv2.destroyAllWindows()

print(f"Total vehicles counted: {int(car_count / 4)}")
print("Details of vehicle detections saved to Excel.")



0: 384x640 2 persons, 4 cars, 435.9ms
Speed: 0.0ms preprocess, 435.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 cars, 439.9ms
Speed: 0.0ms preprocess, 439.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 cars, 425.3ms
Speed: 0.0ms preprocess, 425.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 5 cars, 441.6ms
Speed: 0.0ms preprocess, 441.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 5 cars, 383.1ms
Speed: 0.0ms preprocess, 383.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 5 cars, 391.0ms
Speed: 0.0ms preprocess, 391.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 5 cars, 400.2ms
Speed: 0.0ms preprocess, 400.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 5 cars, 1 truck, 3